The objective of the project is to build a drivers score card that can aid a business to measure the drivers performance based on depot departure, Customer TAT and Depot return. The aim is to create a weighted average per driver and give them intensive based on the grading system. ghhhhh

Importing all the libraries and Data that will be neccessary for the analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

file_path = r'C:\Users\SthembisoM\Desktop\Karl\Driver score card\Trackmatic details.csv'

df = pd.read_csv(file_path)

In [ ]:
print(df.columns)

Data Exploration: Check missing values and duplicates on the dataframe

In [ ]:
duplicate_rows = df[df.duplicated()]
if not duplicate_rows.empty:
    print("Duplicate rows found:")
    print(duplicate_rows)
else:
    print("No duplicate rows found in the dataset.")

missing_values = df.isna().sum()
print("Missing values in each column:")
print(missing_values)

Drop missing data because the trackmatic detail dataset doesnt have any shipment numbers.

In [ ]:
df = df.drop(columns=['Shipment'])
df = df.dropna(subset=['Driver', 'Customer'])
missing_values = df.isna().sum()
print("Missing values in each column:")
print(missing_values)

Correct datatypes

In [ ]:

# List of columns with timestamps to convert
timestamp_columns = ['Requested Delivery Date', 'Planned Route Start', 'Planned Route End', 'Actual Route Start', 'Actual Route End', 'Actual Arrival', 'Planned Departure', 'Actual Departure', 'Planned MST', 'Actual TAT', 'Planned Arrival', 'Actual Route Duration']

# Loop through each timestamp column and convert its data type
for column in timestamp_columns:
    df[column] = pd.to_datetime(df[column], errors='coerce')

# Optional: Check the data types of the columns after conversion
print(df.dtypes)

In [ ]:

unique_route_statuses = df['Route Status'].unique()

# Print the unique strings
for status in unique_route_statuses:
    print(status)

In [ ]:
df = df[~df['Route Status'].isin(['Active', 'Requested'])]
unique_route_statuses = df['Route Status'].unique()

# Print the unique strings
for status in unique_route_statuses:
    print(status)

Drop All Pull Forward or rolled over routes

In [ ]:
condition = df['Actual Route Start'].dt.date != df['Actual Route End'].dt.date

# Use the condition to exclude rows where Planned and Actual Start Dates are different
df = df[~condition]

Drop All sleepouts

In [ ]:
condition = df['Planned Route Start'].dt.date != df['Planned Route End'].dt.date

df = df[~condition]

condition = (df['Planned Route Start'].dt.date == df['Planned Route End'].dt.date)

# Use the condition to exclude rows where Planned Start Date and Planned End Date are different
df = df[condition]

In [ ]:
df = df[df['Planned Route Start'].dt.date == df['Actual Route Start'].dt.date]
df['Departure Time Variance'] = df['Actual Route Start'] - df['Planned Route Start']

df['Departure Time Variance (minutes)'] = df['Departure Time Variance'].dt.total_seconds() / 60
print(df['Departure Time Variance (minutes)'])

In [ ]:
columns_to_drop = [
    'Adj Actual Wait TAT',
    'Adj Actual Off-Load TAT',
    'Adj Actual Doc TAT',
    'Off-Loading TAT per Case (sec)',
    'CSD Planned after 6pm',
    'CSD Arrived after 6pm',
    'PSD Planned after 4pm',
    'PSD Arrived after 4pm',
    'Mall Split',
    'Outlier',
    'Exception']
df = df.drop(columns=columns_to_drop)

In [ ]:

condition = (df['Actual Route Duration'].dt.hour * 60 + df['Actual Route Duration'].dt.minute) >= 60

# Use the condition to exclude rows with a Trip Duration less than 1 hour in the original DataFrame
df = df[condition]

In [ ]:
condition = ~((df['Planned Route Start'].dt.hour >= 7) | (df['Planned Route Start'].dt.hour < 1))

# Use the condition to exclude rows with Actual Start Time between 07:00 and 01:00
df = df[condition]

In [ ]:
condition = ~((df['Actual Route Start'].dt.hour >= 10) | (df['Actual Route Start'].dt.hour < 1))

#Use the condition to exclude rows with Actual Start Time between 13:00 and 01:00
df = df[condition]

In [ ]:
# Find the index of the row with the maximum 'Departure Time Variance (minutes)'
max_idx = df['Departure Time Variance (minutes)'].idxmax()

# Find the index of the row with the minimum 'Departure Time Variance (minutes)'
min_idx = df['Departure Time Variance (minutes)'].idxmin()

# Access the rows with the maximum and minimum values
max_row = df.loc[max_idx]
min_row = df.loc[min_idx]

# Print the rows
print("Row with Maximum Variance:")
print(max_row)

print("\nRow with Minimum Variance:")
print(min_row)


max_variance = df['Departure Time Variance (minutes)'].max()
min_variance = df['Departure Time Variance (minutes)'].min()

print(f"Maximum Variance: {max_variance} minutes")
print(f"Minimum Variance: {min_variance} minutes")

In [ ]:
variance_stats = df['Departure Time Variance (minutes)'].describe()
print(variance_stats)

In [ ]:
dt = df['Departure Time Variance (minutes)']

# Create a histogram with specified number of bins (you can adjust this)
plt.hist(dt, bins=20, edgecolor='k')

# Add labels and a title
plt.xlabel('Departure Time Variance (minutes)')
plt.ylabel('Frequency')
plt.title('Histogram of Departure Time Variance')

# Show the histogram
plt.show()

Based on these statistics:

The mean departure time variance being positive (8.38 minutes) suggests that, on average, departures tend to be slightly later than planned.

The standard deviation being relatively large (47.49 minutes) indicates a wide spread of departure time variances, with some instances of significant delays and early departures.

The presence of negative values (e.g., minimum of -255.63 minutes) indicates instances where departures occurred earlier than planned.

The maximum departure time variance of 478.35 minutes suggests that there are cases of substantial delays in departures.

In conclucsion I am satisfied with the data quality.

Creating Category for dispatching pattern

In [ ]:
# Convert 'Planned Route Start' and 'Actual Route Start' columns to datetime format
df['Planned Route Start'] = pd.to_datetime(df['Planned Route Start'])
df['Actual Route Start'] = pd.to_datetime(df['Actual Route Start'])

# Filter the DataFrame based on date comparison
df = df[df['Planned Route Start'].dt.date == df['Actual Route Start'].dt.date]

# Continue with the calculations and categorizations

# Filter the DataFrame based on date comparison
#df = df[df['Planned Route Start'].dt.date == df['Actual Route Start'].dt.date]

# Calculate 'Departure Time Variance' and 'Departure Time Variance (minutes)'
df['Departure Time Variance'] = df['Actual Route Start'] - df['Planned Route Start']
df['Departure Time Variance (minutes)'] = df['Departure Time Variance'].dt.total_seconds() / 60

# Define the categorization function
def categorize_departure_variance(minutes):
    if minutes <= -120:
        return 'Dispatched 2hrs+ Earlier'
    elif minutes > -120 and minutes <= -60:
        return 'Dispatched 1-2 hrs early'
    elif minutes > -60 and minutes <= -30:
        return 'Dispatched 30min-1hr early'
    elif minutes > -30 and minutes <= 30:
        return 'On Time'
    elif minutes > 30 and minutes <= 60:
        return 'Dispatched 30min-1hr late'
    elif minutes > 60 and minutes < 120:
        return 'Dispatched 1hr-2hrs late'
    else:
        return 'Dispatched 2hrs+ late'

# Apply the custom function to create a new column
df['Dispatch Variance Category'] = df['Departure Time Variance (minutes)'].apply(categorize_departure_variance)

# Display the DataFrame with the new category column
print(df[['Departure Time Variance (minutes)', 'Dispatch Variance Category']])


In [ ]:
print(df.columns)

In [ ]:
# Convert 'Planned Route End' and 'Actual Route End' columns to datetime format
df['Planned Route End'] = pd.to_datetime(df['Planned Route End'])
df['Actual Route End'] = pd.to_datetime(df['Actual Route End'])

# Filter the DataFrame based on date comparison
df = df[df['Planned Route End'].dt.date == df['Actual Route End'].dt.date]

# Calculate 'Depot Return Variance' and 'Depot Return Variance (minutes)'
df['Depot Return Variance'] = df['Actual Route End'] - df['Planned Route End']
df['Depot Return Variance (minutes)'] = df['Depot Return Variance'].dt.total_seconds() / 60

# Define the categorization function
def categorize_departure_variance(minutes):
    if minutes <= -120:
        return 'Returned 2hrs+ Earlier'
    elif minutes > -120 and minutes <= -60:
        return 'Returned 1-2 hrs early'
    elif minutes > -60 and minutes <= -30:
        return 'Returned 30min-1hr early'
    elif minutes > -30 and minutes <= 30:
        return 'On Time'
    elif minutes > 30 and minutes <= 60:
        return 'Returned 30min-1hr late'
    elif minutes > 60 and minutes < 120:
        return 'Returned 1hr-2hrs late'
    else:
        return 'Returned 2hrs+ late'

# Apply the custom function to create a new column
df['Depot Return Variance Category'] = df['Depot Return Variance (minutes)'].apply(categorize_departure_variance)

# Display the DataFrame with the new category column
print(df[['Depot Return Variance (minutes)', 'Depot Return Variance Category']])


In [ ]:
# Convert 'Actual Route End' and 'Actual Route Start' columns to datetime format
df['Actual Route End'] = pd.to_datetime(df['Actual Route End'])
df['Actual Route Start'] = pd.to_datetime(df['Actual Route Start'])

# Convert 'Actual TAT' and 'Planned MST' columns to datetime
df['Actual TAT'] = pd.to_datetime(df['Actual TAT'], errors='coerce')
df['Planned MST'] = pd.to_datetime(df['Planned MST'], errors='coerce')

# Filter the DataFrame to include only rows where 'Actual Route End' and 'Actual Route Start' have the same date
df = df[df['Actual Route End'].dt.date == df['Actual Route Start'].dt.date]

# Calculate 'TAT Variance' and 'TAT Variance (Minutes)'
df['TAT Variance'] = df['Actual TAT'] - df['Planned MST']
df['TAT Variance (Minutes)'] = df['TAT Variance'].dt.total_seconds() / 60

# Filter the DataFrame to keep only rows with TAT Variance greater than or equal to 20 minutes
df = df[df['TAT Variance (Minutes)'] >= 20]

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

def categorize_TA_variance(minutes):
    if minutes > 0 and minutes <= 30:
        return '0 <TAT <=30'
    elif minutes > 30 and minutes <= 60:
        return '30 <TAT <=60'
    elif minutes > 60 and minutes <= 120:
        return '60 <TAT <=120'
    elif minutes > 120 and minutes <= 180:
        return '120 <TAT <=180'
    elif minutes > 180 and minutes <= 240:
        return '180 <TAT <=240'
    else:
        return 'TAT 4hrs +'

# Apply the custom function to create a new column
df['TAT Variance Category'] = df['TAT Variance (Minutes)'].apply(categorize_TA_variance)

print(df[['TAT Variance Category', 'TAT Variance (Minutes)']])






In [ ]:
variance_stats = df['TAT Variance (Minutes)'].describe()
print(variance_stats)

In [ ]:
print(df.columns)

In [ ]:
dt = df[['TAT Variance (Minutes)']]

# Create a histogram with specified number of bins (you can adjust this)
plt.hist(dt, bins=20, edgecolor='k')

# Add labels and a title
plt.xlabel(['TAT Variance (Minutes)'])
plt.ylabel('Frequency')
plt.title('Histogram of TAT Variance')

# Show the histogram
plt.show()

In [ ]:
print(df.columns)

In [ ]:
df['Driver_Score_Dispatch'] = df['Dispatch Variance Category'].apply(
    lambda x: 3 if x == 'Dispatched 2hrs+ Earlier' else
    (5 if x == 'Dispatched 1-2 hrs early' else
    (10 if x == 'Dispatched 30min-1hr early' else
     (10 if x == 'On Time' else
      (5 if x == 'Dispatched 30min-1hr late' else
       (3 if x == 'Dispatched 1hr-2hrs late' else 1))
    )
    )
))

driver_Dispatch_performance = df.groupby('Driver').agg(
    Number_of_Routes=pd.NamedAgg(column='Dispatch Variance Category', aggfunc='count'),
    Driver_Score_Dispatch=pd.NamedAgg(column='Driver_Score_Dispatch', aggfunc='sum')
)

# Calculate the driver score as a percentage
driver_Dispatch_performance['Driver_Score_Percentage'] = (driver_Dispatch_performance['Driver_Score_Dispatch'] /
                                                  (driver_Dispatch_performance['Number_of_Routes']*10))*100

# Sort the results by the number of routes in descending order
driver_Dispatch_performance = driver_Dispatch_performance.sort_values(by='Number_of_Routes', ascending=False)

# Display the results
print(driver_Dispatch_performance)

In [ ]:
df['Driver_Score_Return'] = df['Depot Return Variance Category'].apply(
    lambda x: 10 if x == 'Returned 2hrs+ Earlier' else
    (10 if x == 'Returned 1-2 hrs early' else
    (10 if x == 'Returned 30min-1hr early' else
     (10 if x == 'On Time' else
      (3 if x == 'Returned 30min-1hr late' else
       (1 if x == 'Returned 1hr-2hrs late' else 0))
    )
    )
))

driver_Return_performance = df.groupby('Driver').agg(
    Number_of_Routes=pd.NamedAgg(column='Depot Return Variance Category', aggfunc='count'),
    Driver_Score_Return=pd.NamedAgg(column='Driver_Score_Return', aggfunc='sum')
)

# Calculate the driver score as a percentage
driver_Return_performance['Driver_Return_Score_Percentage'] = (driver_Dispatch_performance['Driver_Score_Dispatch'] /
                                                  (driver_Dispatch_performance['Number_of_Routes']*10))*100

# Sort the results by the number of routes in descending order
driver_Return_performance = driver_Return_performance.sort_values(by='Number_of_Routes', ascending=False)

# Display the results
print(driver_Return_performance)

 

In [ ]:
df['Driver_TAT_Score'] = df['TAT Variance Category'].apply(
    lambda x: 10 if x == '0 <TAT <=30' else
    (8 if x == '30 <TAT <=60' else
    (5 if x == '60 <TAT <=120' else
     (3 if x == '120 <TAT <=180' else
      (1 if x == '180 <TAT <=240' else 0)
    )
    )
))

driver_TAT_performance = df.groupby('Driver').agg(
    Number_of_Routes=pd.NamedAgg(column='TAT Variance Category', aggfunc='count'),
    Driver_Score_TAT=pd.NamedAgg(column='Driver_TAT_Score', aggfunc='sum')
)

# Calculate the driver score as a percentage
driver_TAT_performance['Driver_TAT_Score_Percentage'] = (driver_TAT_performance['Driver_Score_TAT'] /
                                                  (driver_TAT_performance['Number_of_Routes']*10))*100

# Sort the results by the number of routes in descending order
driver_TAT_performance = driver_TAT_performance.sort_values(by='Number_of_Routes', ascending=False)

# Display the results
print(driver_TAT_performance, driver_Return_performance, driver_Dispatch_performance)


In [ ]:
# Calculate the weighted score for each driver in each performance category
driver_TAT_performance['Weighted_Score_TAT'] = (driver_TAT_performance['Driver_TAT_Score_Percentage'] * 0.4)
driver_Return_performance['Weighted_Score_Return'] = (driver_Return_performance['Driver_Return_Score_Percentage'] * 0.3)
driver_Dispatch_performance['Weighted_Score_Dispatch'] = (driver_Dispatch_performance['Driver_Score_Percentage'] * 0.3)

# Combine the three DataFrames into one, using the 'Driver' column as the index
driver_combined_performance = pd.concat([driver_TAT_performance, driver_Return_performance, driver_Dispatch_performance], axis=1)

# Calculate the total weighted score by summing the scores from each category
driver_combined_performance['Total_Weighted_Score'] = (
    driver_combined_performance['Weighted_Score_TAT'] +
    driver_combined_performance['Weighted_Score_Return'] +
    driver_combined_performance['Weighted_Score_Dispatch']
)

# Select the relevant columns for the final DataFrame
weighted_score_df = driver_combined_performance[['Number_of_Routes', 'Total_Weighted_Score']]

# Sort drivers by their total weighted scores in descending order
weighted_score_df = weighted_score_df.sort_values(by='Total_Weighted_Score', ascending=False)

# Reset the index to have a clean index without duplicate driver names
weighted_score_df.reset_index(inplace=True)

# Display the results
print(weighted_score_df)
